In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
cd drive

In [0]:
cd My Drive

In [0]:
cd Colab Notebooks

In [0]:
import pickle
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks

In [0]:
img_width, img_height = 128, 128
batch_size = 32
samples_per_epoch = 2500
epochs = 30
validation_steps = 300
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 6
lr = 0.0004

classifier = Sequential()
classifier.add(Convolution2D(32, kernel_size=(3, 3),padding='same',input_shape=(128, 128, 3)))
classifier.add(Activation('relu'))
classifier.add(Convolution2D(64, (3, 3)))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.25))

classifier.add(Convolution2D(64,(3, 3), padding='same'))
classifier.add(Activation('relu'))
classifier.add(Convolution2D(64, 3, 3))
classifier.add(Activation('relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.25))

classifier.add(Flatten())
classifier.add(Dense(512))
classifier.add(Activation('relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(6))
classifier.add(Activation('softmax'))

classifier.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=lr),
              metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
  "TRAIN2",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
  "TEST2",
  target_size=(img_height, img_width),
  batch_size=batch_size,
  class_mode='categorical')

classifier.fit_generator(
  train_generator,
  samples_per_epoch = samples_per_epoch,
  epochs=epochs,
  validation_data=validation_generator,
  validation_steps=validation_steps)


with open('classifier.pickle', 'wb') as handle:
    pickle.dump(classifier, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [0]:
from keras.preprocessing import image
import numpy as np

with open('classifier.pickle', 'rb') as handle:
    classifier = pickle.load(handle)

test_image = image.load_img('00009.jpg', target_size = (128, 128))

test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)


print(result)